# PARTIE 1 - TUTO COURS

In [ ]:
# Importation des librairie

import pandas as pd
import geopandas as gd
import matplotlib.pyplot as plt
import datetime

df = pd.read_csv("patient-data-anonymisation-exercise.csv")
df.head()

In [ ]:
# Voir la liste des fields des données 

df.info()

In [ ]:
# Utilisons aussi geopandas et regardons l'emplacement physique de nos données

gdf = gd.GeoDataFrame(df, crs="EPSG:4326", geometry=gd.points_from_xy(df.LON, df.LAT))

In [ ]:
# reproject
gdf = gdf.to_crs(epsg=3857)
ax = gdf.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax=ax, source=ctx.providers.Stamen.TonerLite, zoom=12)
ax.set_axis_off()

In [ ]:
# Préciser les colonnes pour l'éloignement
suppression = ["SSN", "DRIVERS", "PASSPORT", "FIRST", "LAST", "MAIDEN", "ADDRESS", "ZIP"]
# Et les laisser tomber
df.drop(suppression, axis=1, inplace=True)

In [ ]:
# Ecrivons une class Python à exécuter pendant que nous anonymisons nos données

import uuid
import numpy as np
from collections import defaultdict


class Pseudonymise:
    
    def __init__(self, sigma=0.001):
        # Initialiser un dictionnaire par défaut, cela crée un élément de dictionnaire par défaut s'il n'existe pas
        # Nous utilisons cette méthode pour garantir l'intégrité des données tout en les rendant aléatoires
        # http://ikigomu.com/?p=28
        self.sigma = sigma
        self.mu = 0 # Nous voulons nous écarter de la vérité
        # Pseudo-patient dict
        self.pp = defaultdict(lambda: {"uuid":str(uuid.uuid4()),
                                       "lat": np.random.normal(self.mu, self.sigma),
                                       "lon": np.random.normal(self.mu, self.sigma)
                                      })
        
    def create_data(self, identities):
        """
        Pour chaque identité unique produire un UUID unique, et un gaussien randomisé "LAT" et "LON".

        Parameters
        ----------
        identities: liste des chaînes de caractères

        Returns
        -------
        dict
            Chaque entrée de dict contient un enregistrement contenant un "uuid" et un modificateur pour "lat" et "lon".
        """
        for _id in identities:
            self.pp[_id]
        return self.pp
    
    def redact(self, row):
        """
        Pour une ligne donnée dans un dataframe, retournez la version pseudonymisée de "PATIENT_ID", "LAT", "LON".
        
        Parameters:
        row : Ligne de la trame de données
        
        row:
        DataFrame tranche de ligne 
        """
        return [
            self.pp[row["PATIENT_ID"]]["uuid"],
            row["LAT"] + self.pp[row["PATIENT_ID"]]["lat"],
            row["LON"] + self.pp[row["PATIENT_ID"]]["lon"],
        ]

p = Pseudonymise()
pp_data = p.create_data(df["PATIENT_ID"])
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
fields = ["PATIENT_ID", "LAT", "LON"]
df[fields] = df[fields].apply(p.redact, axis=1, result_type="expand")

In [ ]:
# Lancons la visualisation dans GeoPandas

gdf = gd.GeoDataFrame(df, crs="EPSG:4326", geometry=gd.points_from_xy(df.LON, df.LAT))
gdf = gdf.to_crs(epsg=3857)
ax = gdf.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax=ax, source=ctx.providers.Stamen.TonerLite, zoom=12)
ax.set_axis_off()

# PARTIE 2 - L'AGGREGATION 

In [ ]:
# Convertir en date 
df = df.copy()

for c in ["BIRTHDATE"]:
    df[c] = df[c].apply(lambda x: pd.to_datetime(x, errors="coerce", dayfirst=True).date())

def annee_uniquement(value):
    return value.year

df["BIRTHDATE"]=df["BIRTHDATE"].apply(annee_uniquement)

In [ ]:
# Ne montrer que l'année

A=df["BIRTHDATE"]
A.head(30)

In [ ]:
# Agréger par chaque tranche d'age les raisons de la visite 

groupe1 = df.groupby(["BIRTHDATE", 'DESCRIPTION'])
B = groupe1.aggregate(np.sum)
B.drop(B.iloc[:,:],1,inplace=True)
B.head(30)


In [ ]:
 # Calculer pour chaque tranche d'âge le total de tous les coûts des sinistres¶
    
groupe2 = df.groupby(["BIRTHDATE", 'TOTAL_CLAIM_COST'])
C = groupe2.aggregate(np.sum)
C.drop(C.iloc[:,:],1,inplace=True)
C.head(30)


In [ ]:
# Calculons pour chaque tranche d'âge, la médiane de tous les coûts des sinistres

groupe3 = df.groupby(["BIRTHDATE", 'TOTAL_CLAIM_COST'])
D = groupe3.aggregate(np.median)
D.head(30)